In [5]:
from datetime import datetime
import pickle
import uuid
import datetime

# Elastic Search

In [6]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(hosts="http://localhost:9200" , verify_certs=False)

In [3]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}

In [4]:
resp = es.index(index="test-index", id=1, document=doc)


In [5]:
print(resp['result'])

resp = es.get(index="test-index", id=1)
print(resp['_source'])

es.indices.refresh(index="test-index")

resp = es.search(index="test-index", query={"match_all": {}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

created
{'author': 'kimchy', 'text': 'Elasticsearch: cool. bonsai cool.', 'timestamp': '2022-10-14T08:23:06.496072'}
Got 1 Hits:
2022-10-14T08:23:06.496072 kimchy: Elasticsearch: cool. bonsai cool.


# Milvus

In [ ]:
# https://raw.githubusercontent.com/milvus-io/pymilvus/v1.1.2/examples/example.py
# This program demos how to connect to Milvus vector database, 
# create a vector collection,
# insert 10 vectors, 
# and execute a vector similarity search.

import random

from milvus import Milvus, IndexType, MetricType, Status

# Milvus server IP address and port.
# You may need to change _HOST and _PORT accordingly.
_HOST = '127.0.0.1'
_PORT = '19530'  # default value
# _PORT = '19121'  # default http value

# Vector parameters
_DIM = 8  # dimension of vector

_INDEX_FILE_SIZE = 32  # max file size of stored index


def milvus_test():
    # Specify server addr when create milvus client instance
    # milvus client instance maintain a connection pool, param
    # `pool_size` specify the max connection num.
    milvus = Milvus(_HOST, _PORT)

    # Create collection demo_collection if it dosen't exist.
    collection_name = 'example_collection_'

    status, ok = milvus.has_collection(collection_name)
    if not ok:
        param = {
            'collection_name': collection_name,
            'dimension': _DIM,
            'index_file_size': _INDEX_FILE_SIZE,  # optional
            'metric_type': MetricType.L2  # optional
        }

        milvus.create_collection(param)

    # Show collections in Milvus server
    _, collections = milvus.list_collections()

    # Describe demo_collection
    _, collection = milvus.get_collection_info(collection_name)
    print(collection)

    # 10000 vectors with 128 dimension
    # element per dimension is float32 type
    # vectors should be a 2-D array
    vectors = [[random.random() for _ in range(_DIM)] for _ in range(10)]
    print(vectors)
    # You can also use numpy to generate random vectors:
    #   vectors = np.random.rand(10000, _DIM).astype(np.float32)

    # Insert vectors into demo_collection, return status and vectors id list
    status, ids = milvus.insert(collection_name=collection_name, records=vectors)
    if not status.OK():
        print("Insert failed: {}".format(status))

    # Flush collection  inserted data to disk.
    milvus.flush([collection_name])
    # Get demo_collection row count
    status, result = milvus.count_entities(collection_name)

    # present collection statistics info
    _, info = milvus.get_collection_stats(collection_name)
    print(info)

    # Obtain raw vectors by providing vector ids
    status, result_vectors = milvus.get_entity_by_id(collection_name, ids[:10])

    # create index of vectors, search more rapidly
    index_param = {
        'nlist': 2048
    }

    # Create ivflat index in demo_collection
    # You can search vectors without creating index. however, Creating index help to
    # search faster
    print("Creating index: {}".format(index_param))
    status = milvus.create_index(collection_name, IndexType.IVF_FLAT, index_param)

    # describe index, get information of index
    status, index = milvus.get_index_info(collection_name)
    print(index)

    # Use the top 10 vectors for similarity search
    query_vectors = vectors[0:10]

    # execute vector similarity search
    search_param = {
        "nprobe": 16
    }

    print("Searching ... ")

    param = {
        'collection_name': collection_name,
        'query_records': query_vectors,
        'top_k': 1,
        'params': search_param,
    }

    status, results = milvus.search(**param)
    if status.OK():
        # indicate search result
        # also use by:
        #   `results.distance_array[0][0] == 0.0 or results.id_array[0][0] == ids[0]`
        if results[0][0].distance == 0.0 or results[0][0].id == ids[0]:
            print('Query result is correct')
        else:
            print('Query result isn\'t correct')

        # print results
        print(results)
    else:
        print("Search failed. ", status)

    # Delete demo_collection
    status = milvus.drop_collection(collection_name)


milvus_test()

# weaviate

In [10]:
#!pip install weaviate-client==3.8.0

In [12]:
import weaviate


In [13]:
def generate_uuid(class_name: str, identifier: str,
                  test: str = 'teststrong') -> str:
    """ Generate a uuid based on an identifier
    :param identifier: characters used to generate the uuid
    :type identifier: str, required
    :param class_name: classname of the object to create a uuid for
    :type class_name: str, required
    """
    test = 'overwritten'
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, class_name + identifier))

def log(i: str) -> str:
    """ A simple logger
    :param i: the log message
    :type i: str
    """
    now = datetime.datetime.utcnow()
    print(now, "| " + str(i))

In [17]:
client = weaviate.Client("http://localhost:8081")
print("Client created")

Client created


In [18]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens') #, Initially load using this, then start using pickle to save time.

In [19]:
# from sentence_transformers import SentenceTransformer
# # sbert_model = SentenceTransformer('bert-base-nli-mean-tokens'), Initially load using this, then start using pickle to save time.
# with open("sbert",'rb') as f:
#     sbert_model = pickle.load(f)

print("sbert loaded")

# I am adding the texts in this list,
# We can also add sentences of a large text individually to get more precise results when we query.
documents = [
    '''Taj mahal is an immense mausoleum of white marble, built in Agra between 1631 and 1648 by order of the Mughal emperor Shah Jahan in memory of his favourite wife, the Taj Mahal is the jewel of Muslim art in India and one of the universally admired masterpieces of the world's heritage.''',
    '''The Statue of Liberty is a 305-foot (93-metre) statue located on Liberty Island in Upper New York Bay, off the coast of New York City. The statue is a personification of liberty in the form of a woman. She holds a torch in her raised right hand and clutches a tablet in her left.''',
    '''The Statue of Liberty was sculpted between 1875 and 1884 under the direction of French sculptor Frédéric-Auguste Bartholdi, who began drafting designs in 1870. Bartholdi and his team hammered roughly 31 tons of copper sheets onto a steel frame. Before being mounted on its current pedestal, the statue stood over 151 feet (46 metres) tall and weighed 225 tons.''',
    '''Badminton is a racquet sport played using racquets to hit a shuttlecock across a net. Although it may be played with larger teams, the most common forms of the game are "singles" (with one player per side) and "doubles" (with two players per side). Badminton is often played as a casual outdoor activity in a yard or on a beach; formal games are played on a rectangular indoor court. Points are scored by striking the shuttlecock with the racquet and landing it within the opposing side's half of the court.''',
    '''James Bond is a fictional character created by novelist Ian Fleming in 1953.''',
    '''A British secret agent working for MI6 under the codename 007, he has been portrayed on film by actors Sean Connery, David Niven, George Lazenby, Roger Moore, Timothy Dalton, Pierce Brosnan and Daniel Craig in twenty-seven productions.'''
]

# A dictionary to store the document and its feature vector (the vector generated by SBERT)
doc_and_vec = {}

def giveVector(texts):
    # this function returns the vector using SBERT
    return sbert_model.encode(texts)

vectors = giveVector(documents)

for doc,vec in zip(documents,vectors):
    doc_and_vec[doc] = vec

print("vectors formed")

client.schema.delete_all()
class_obj = {
    "class": "Post",
    "vectorizer": "none", # we are providing the vectors ourselves through our SBERT model, so this field is none
    "properties": [{
        "name": "content",
        "dataType": ["text"],
    }]
}

client.schema.create_class(class_obj)
print("Schema class created")

for doc,vec in doc_and_vec.items():
    data_obj = {
    "content": doc
    }
    client.data_object.create(
    data_obj,
    "Post",
    generate_uuid('Post',doc),
    vector = vec,
    )
print("Finished importing data")

def process_query(vec):
    nearVector = {"vector": vec}
    res = client.query.get("Post", ["content", "_additional {certainty}"]).with_near_vector(nearVector).do()
    print(res)
    print("------------------------------------------------------------------------------------------------")
    print("-----------------------------------Most similar text -------------------------------------------")
    print(res['data']['Get']['Post'][0]['content'])
    print("------------------------------------------------------------------------------------------------")
    print(res['data']['Get']['Post'][1]['content'])
    print("------------------------------------------------------------------------------------------------")

    


sbert loaded
vectors formed
Schema class created
Finished importing data


In [20]:
query ="american tourist destination"
query_vec = sbert_model.encode(query)
process_query(query_vec)


{'data': {'Get': {'Post': [{'_additional': {'certainty': 0.6700640916824341}, 'content': "Taj mahal is an immense mausoleum of white marble, built in Agra between 1631 and 1648 by order of the Mughal emperor Shah Jahan in memory of his favourite wife, the Taj Mahal is the jewel of Muslim art in India and one of the universally admired masterpieces of the world's heritage."}, {'_additional': {'certainty': 0.6459582149982452}, 'content': 'A British secret agent working for MI6 under the codename 007, he has been portrayed on film by actors Sean Connery, David Niven, George Lazenby, Roger Moore, Timothy Dalton, Pierce Brosnan and Daniel Craig in twenty-seven productions.'}, {'_additional': {'certainty': 0.6445140540599823}, 'content': 'James Bond is a fictional character created by novelist Ian Fleming in 1953.'}, {'_additional': {'certainty': 0.6366377472877502}, 'content': 'The Statue of Liberty is a 305-foot (93-metre) statue located on Liberty Island in Upper New York Bay, off the coa